In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import jax
from interp.model.model_loading import load_model, get_dataset_mean_activations
from interp.tools.grad_modify_query import  ModifierCollectionTreeNode, Query, TargetConf, run_query
from interp.tools.grad_modify_query_items import ItemConf, MulConf
from interp.tools.grad_modify_query_utils import compose_trees
from interp.tools.data_loading import get_val_seqs
import numpy as np
import jax.numpy as jnp
from functools import partial
import plotly.express as px
import plotly.graph_objects as go

# model_name = "GPT_canon"
model_name = "GPT_canon"
# model_name = "jan11_gpt_2l_canon"
# model_name = "apr5_2l"
# model_name = "jan5_attn_only_two_layers"
model, params, tokenizer = load_model(model_name, models_dir="/home/ubuntu/interpretability_models_jax",
load_params=True,)
from interp.model.canon import canonicalize
# params_canon = canonicalize(model,params, emb=None)


In [ ]:
import interp.cui as cui
import jax.numpy as jnp
import jax
from interp.ui.very_named_tensor import VeryNamedTensor
# import interp.tensor_makers.atttention_weighted as aw
# import interp.tensor_makers.correct_logprobs as cp
# import interp.tensor_makers.logprobs as mp
await cui.init(port=6789)

In [ ]:
# from interp.model.canon import auto_canonicalize
# params = auto_canonicalize(model,params)
# from interp.model.model_loading import save_model
# save_model(model,params,"gpt2_canon","GPT","canonicalized gpt2")


In [ ]:
import interp.ui.dataset_search as ds
text = "Yesterday John Von Neumann walked around the park with his dog. John Von Neumann was"
# text1 = "On Saturday, Mr Smith went to the mall."
# text2 = "On Saturday, Mrs Smith went to the mall."
token_ids = tokenizer(text, padding=False, return_tensors="jax")["input_ids"]

In [ ]:
from interp.tools.log import LoggerCacheAll, MutLogCache,LogInfo

def normalize(x):
    return x/jnp.expand_dims(jnp.linalg.norm(x,axis=-1),-1)

def get_mlp_stuff(layer):
    i_kernel = jnp.transpose(params["params"][f"blocks_{layer}"]["linear1"]["kernel"])
    i_bias = params["params"][f"blocks_{layer}"]["linear1"]["bias"]
    o_kernel = params["params"][f"blocks_{layer}"]["linear2"]["kernel"]
    return i_kernel,i_bias,o_kernel
mlps = [get_mlp_stuff(x) for x in range(model.num_layers)]
ovs = model.bind(params).get_ov_combined_mats_all_layers()
os = model.bind(params).get_o_mats_all_layers()
token_embedding = params["params"]["embedding"]["token_embedding"]["embedding"]
pos_embedding = params["params"]["embedding"]["position_embedding"]["embedding"]
# tu,ts,tok_svd_transform = jnp.linalg.svd(token_embedding)
# pu,ps,pos_svd_transform = jnp.linalg.svd(pos_embedding)
mlps = [jnp.stack([mlp[k] for mlp in mlps],axis=0) for k in range(len(mlps[0]))]
print([x.shape for x in mlps])

In [ ]:
# o_argsort = jnp.argsort(jnp.linalg.norm(mlps[2],axis=-1),axis=-1)
# i_by_mag = jnp.stack([mlps[0][l][o_argsort[l]] for l in range(model.num_layers)])
# o_by_mag = jnp.stack([mlps[2][l][o_argsort[l]] for l in range(model.num_layers)])
i_by_mag = mlps[0]
o_by_mag = mlps[2]

# print(i_by_mag.shape,o_by_mag.shape,jnp.argsort(jnp.linalg.norm(mlps[2],axis=-1),axis=-1).shape)
dots = jnp.einsum("lni,lmi->lnm",i_by_mag,o_by_mag)
dots_i = jnp.einsum("lni,lmi->lnm",i_by_mag,i_by_mag)
dots_o = jnp.einsum("lni,lmi->lnm",o_by_mag,o_by_mag)
sims = jnp.einsum("lni,lmi->lnm",normalize(i_by_mag),normalize(o_by_mag))
sims_i = jnp.einsum("lni,lmi->lnm",normalize(i_by_mag),normalize(i_by_mag))
sims_o = jnp.einsum("lni,lmi->lnm",normalize(o_by_mag),normalize(o_by_mag))
tvnt = lambda x,t: vnt_guessing_shit_model_tokens(x,model,np.array(range(500)),title=t)
from interp.ui.very_named_tensor import vnt_guessing_shit_model_tokens
# await cui.show_tensors(tvnt(sims,"io"),tvnt(sims_i,"ii"),tvnt(sims_o,"oo"))
iosim_argsort = jnp.argsort(sims[:,np.arange(model.hidden_size*4),np.arange(model.hidden_size*4)],axis=-1)
print(iosim_argsort)


In [ ]:
def tothing(x):
    return jnp.mean(jnp.mean(x.at[:,np.arange(model.hidden_size*4),np.arange(model.hidden_size*4)].set(0),axis=-1),axis=-1)
[]
await cui.show_tensors(tvnt(sims[:,np.arange(model.hidden_size*4),np.arange(model.hidden_size*4)],"sims"),tvnt(mlps[1],"bias"),tvnt(dots[:,np.arange(model.hidden_size*4),np.arange(model.hidden_size*4)],"dots"),name="diag")



In [ ]:
def plot_direction_segment_through_mlp(start,end,layer):
    vals = model.bind(params).blocks[layer].mlp(jnp.expand_dims(jnp.expand_dims(jnp.linspace(0,1,num=100,dtype=start.dtype),axis=1)*(end-start)+start,axis=0))
    result = jnp.einsum("bsh,h->s",vals,(end-start))
    return result
# px.line(plot_direction_segment_through_mlp(-mlps[0][0,1168],mlps[0][0,1166] ,0))

# px.line(plot_direction_segment_through_mlp(-mlps[0][0,906],mlps[0][0,906] ,9))


In [ ]:
from tqdm import tqdm
from interp.model.blocks import gelu
def plot_mlp_input_graphs():
    scale=1
    vals = []
    for layer in tqdm(range(model.num_layers)):
        ivectors = mlps[0][layer]
        out = model.bind(params).blocks[layer].mlp(jnp.expand_dims(jnp.linspace(0,1,num=50,dtype=ivectors.dtype),axis=1)*jnp.expand_dims(ivectors*2,axis=1)*scale+jnp.expand_dims(-ivectors*scale,axis=1))
        vals.append( jnp.einsum("nsh,nh->ns",out,ivectors*scale))
    result = jnp.stack(vals)
    return result
def plot_mlp_input_graphs_nonorm():
    scale=1
    vals = []
    for layer in tqdm(range(model.num_layers)):
        ivectors = mlps[0][layer]
        input = jnp.expand_dims(jnp.linspace(0,1,num=50,dtype=ivectors.dtype),axis=1)*jnp.expand_dims(ivectors*2,axis=1)*scale+jnp.expand_dims(-ivectors*scale,axis=1)
        block = model.bind(params).blocks[layer]
        out = block.linear2(gelu(block.linear1(input)))
        vals.append( jnp.einsum("nsh,nh->ns",out,ivectors*scale))
    result = jnp.stack(vals)
    return result
def plot_mlp_input_graphs_single_neuron():
    scale=1
    vals = []
    for layer in tqdm(range(model.num_layers)):
        ivectors = mlps[0][layer]
        input = jnp.expand_dims(jnp.linspace(0,1,num=50,dtype=ivectors.dtype),axis=1)*jnp.expand_dims(ivectors*2,axis=1)*scale+jnp.expand_dims(-ivectors*scale,axis=1) # neuron x h
        block_params = model.bind(params).blocks[layer].variables["params"]
        neurons = gelu(jnp.einsum("hn,nxh->nx",block_params["linear1"]["kernel"],input)+jnp.expand_dims(block_params["linear1"]["bias"],axis=1))
        print(block_params["linear2"]["kernel"].shape,neurons.shape,block_params["linear2"]["bias"].shape)
        out = jnp.einsum("nh,nx->nxh",block_params["linear2"]["kernel"],neurons)+block_params["linear2"]["bias"]
        out_in_dir = jnp.einsum("nxh,hn->nx",out, block_params["linear1"]["kernel"])*scale
        vals.append( out_in_dir)
    result = jnp.stack(vals)
    return result
single_neuron = plot_mlp_input_graphs_single_neuron()
nonorm = plot_mlp_input_graphs_nonorm()
withnorm = plot_mlp_input_graphs()
stadked = jnp.stack([nonorm,single_neuron])


stadked_byiosort = jnp.stack([stadked[:,l,iosim_argsort[l]] for l in range(model.num_layers)],axis=1)

In [ ]:
print(stadked.shape,stadked_byiosort.shape)
await cui.show_tensors(tvnt(stadked_byiosort[:,:,np.array([0,1,2,3])],"top"),tvnt(stadked_byiosort[:,:,np.array([1500,1501,1502,1503])],"middle"),tvnt(stadked_byiosort[:,:,np.array([-1,-2,-3,-4])],"bottom"),name="thang")



In [ ]:
model.bind(params).blocks[0].linear1.variables["params"]["kernel"]

In [ ]:

print([[tokenizer.decode([x]),i] for i,x in enumerate(token_ids[0])])
embed_mask = np.zeros((token_ids.shape[1],),dtype=np.bool8)
embed_mask[2] = 1
head_mask = np.zeros((model.num_layers,model.num_heads, token_ids.shape[1]),dtype=np.bool8)
head_mask[0,0,3] = 1
head_mask[1,4,14] = 1
mlp_mask = np.zeros((model.num_layers,model.hidden_size*4,token_ids.shape[1]),dtype=np.bool8)

mlp_mask_tokenless = np.zeros((model.num_layers,model.hidden_size*4), dtype=np.bool8)
head_mask_tokenless = np.zeros((model.num_layers,model.num_heads), dtype=np.bool8)
head_mask_tokenless[0,7] = True
head_mask_tokenless[1,7] = True
# ds.embeds_through_mask_ablation(model,params,token_ids,embed_mask,head_mask,mlp_mask)
ds.between_mask_ablation_unjit(model,params, token_ids,head_mask_tokenless,mlp_mask_tokenless,model_name)

In [ ]:
jnp.array(0)

In [ ]:
from interp.ui.attribution_backend import AttributionBackend
backend = AttributionBackend(model, params, tokenizer, text)
vnt_start = backend.startTree(
    {"kind": "logprob", "data": {"seqIdx": 14, "tokString": "Ne", "comparisonTokString": None}}
)
_,_,masks = backend.searchAttributionsFromStart(0.05,True)
print({k:jnp.sum(v) for k,v in masks.items()})
masks_tokenless = {"embeds":np.any(masks["embeds"],axis=-1,keepdims=True),"heads":np.any( masks["heads"],axis=-1,keepdims=True),"mlps":np.any( masks["mlps"],axis=-1,keepdims=True)}
print(masks_tokenless)
ds.embeds_through_mask_ablation(model,params,token_ids,masks["embeds"],masks["heads"],np.expand_dims(masks["mlps"],axis=1))



In [ ]:
import interp.cui as cui
import jax.numpy as jnp
import jax
from interp.ui.very_named_tensor import VeryNamedTensor
# import interp.tensor_makers.atttention_weighted as aw
# import interp.tensor_makers.correct_logprobs as cp
# import interp.tensor_makers.logprobs as mp
await cui.init(port=6789)

In [ ]:
masks_tokenless["embeds"].shape

In [ ]:
await ds.show_top_docs_fn(lambda token_ids: ds.embeds_through_mask_ablation(model,params,token_ids,masks_tokenless["embeds"],masks_tokenless["heads"],masks_tokenless["neurons"]),n_seqs=100,batch_size=1,seq_len=64,model=model)

In [ ]:
from interp.tools.data_loading import get_val_seqs
from interp.model.gpt_model import inference,inference_unjit
from tqdm import tqdm

def collect_mean_activations(model,params, n=11000,batch_size=8,seq_len=512):
    trimmed_n = (n//batch_size)*batch_size
    seqs = get_val_seqs()[:trimmed_n,:seq_len]
    seq_len = seqs.shape[-1]
    
    seqs = seqs.reshape(-1,batch_size,seq_len)
    to_log = [f"blocks.{l}.attention.out_by_head" for l in range(model.num_layers)]
    if model.use_mlp:
        to_log.extend([f"blocks.{l}.mlp.out_by_neuron" for l in range(model.num_layers)])
    to_log=tuple(to_log)
    sums = None
    for seq in tqdm(seqs):
        _,log = inference(model,params,seq,to_log)
        if sums==None:
            sums = {k:jnp.mean(log[k],axis=0) for k in to_log}
        else:
            for k,v in sums.items():
                sums[k] = v+jnp.mean(log[k],axis=0)
    sums = {k:jnp.mean(v/seqs.shape[0],axis=1) for k,v in sums.items()}
    stacked= {"attention":jnp.stack([sums[f"blocks.{l}.attention.out_by_head"] for l in range(model.num_layers)])}
    if model.use_mlp:
        stacked["mlp"]=jnp.stack([sums[f"blocks.{l}.mlp.out_by_neuron"] for l in range(model.num_layers)])
    return stacked
mean_activations = collect_mean_activations(model,params,n=100,batch_size=6)
print([k.shape for k in mean_activations.values()])

In [ ]:
import os
from flax.serialization import msgpack_serialize
def save_all_avg_activations():
    # for model_name in ["jan5_attn_only_two_layers", "jan11_gpt_2l", "dec30_attn_only_two_layers", "GPT2"]:
    for model_name in ["jan11_gpt_2l"]:
        model, params, tokenizer = load_model(model_name)
        mean_activations = collect_mean_activations(model,params,batch_size=1,seq_len=64)
        try:
            os.mkdir(f"/home/ubuntu/interpretability_model_stats/{model_name}")
        except:
            pass
        open(f"/home/ubuntu/interpretability_model_stats/{model_name}/mean_activations","wb").write(msgpack_serialize(mean_activations))
save_all_avg_activations()

In [ ]:

log = LogCacheAll()
model_b(val_seqs[0:1], log=log)

In [ ]:
log['blocks.1.mlp.linear1']

In [ ]:
from copy import deepcopy
A = ds.get_mlp_stats_unjit(model, params, input_ids=val_seqs[1:5])
A.shape

In [ ]:
from dataset_search import *

def get_mlp_grads_unjit(model, params, input_ids_batch,l):

    result = run_query(
        loss_by_token_loggable(model, params, input_ids),
        query=Query(
            targets=[TargetConf("loss.lm_loss", display_name="loss")],
            modifier_collection_tree=ModifierCollectionTreeNode(MulConf(ItemConf(f"blocks.{l}.mlp.linear1"),shape=I[1,1,4*model.hidden_size])),
            use_fwd=False,
        ),
    )
    
    result = result['loss'][:,:,0,0,:]
    return result

dataset_here = get_val_seqs()
batch = dataset_here[:2,:128]
# dataset_batches = dataset_here.reshape(-1, 10, 100)
# print(dataset_batches.shape)
result = get_mlp_grads_unjit(model, params, batch, 0)

In [ ]:
result.shape

In [ ]:
result.shape

In [ ]:
from tqdm import tqdm
from interp.ui.dataset_search import get_mlp_stats

n_seqs = 3000
batch_size = 10
seq_len = 100
k = 5
agg = 'mean'

assert agg in {'mean', 'max'}
dataset_here = get_val_seqs()[: (n_seqs // batch_size) * batch_size, :seq_len]
dataset_batches = dataset_here.reshape(-1, batch_size, seq_len)
result = []
top_k_doc_idx = np.zeros((k, model.num_layers, model.hidden_size*4), dtype=np.int32)
top_k_vals = -np.infty*np.ones((k, model.num_layers, model.hidden_size*4))
for i, batch in tqdm(enumerate(dataset_batches)):
    activations = np.array(get_mlp_stats(model, params, batch), dtype=np.float16)
    if agg == 'mean':
        per_batch_vals = activations.mean(axis=-2)
    elif agg == 'max':
        per_batch_vals = activations.max(axis=-2)
    else:
        assert False,"agg should be in {'mean', 'max'} (and an assertion should have caught it earlier)"

    
    batch_and_top_k_vals = np.concatenate((per_batch_vals, top_k_vals), axis=0)
    bk_top_idx = batch_and_top_k_vals.argsort(axis=0)[-k:]
    top_k_vals = np.take_along_axis(batch_and_top_k_vals, bk_top_idx, axis=0)
    
    new_k_idx, l_idx, n_idx = np.where(bk_top_idx >= batch_size)
    top_k_doc_idx[new_k_idx, l_idx, n_idx] = top_k_doc_idx[bk_top_idx[new_k_idx, l_idx, n_idx] - batch_size, l_idx, n_idx]
    top_k_doc_idx[bk_top_idx < batch_size] = bk_top_idx[bk_top_idx < batch_size] + i*batch_size

top_k_doc_idx, top_k_vals = np.moveaxis(top_k_doc_idx, 0, -1), np.moveaxis(top_k_vals, 0, -1)    
top_k_doc_idx, top_k_vals = np.flip(top_k_doc_idx, axis=-1), np.flip(top_k_vals, axis=-1)


In [ ]:
for i in range(5):
    print(tokenizer.decode(dataset_here[top_k_doc_idx[1,8]][i]))

In [ ]:
from interp.ui.dataset_search import *


def get_mlp_grads_unjit(model, params, input_ids, l):

    result = run_query(
        loss_by_token_loggable(model, params, input_ids),
        query=Query(
            targets=[TargetConf("loss.lm_loss", display_name="loss")],
            modifier_collection_tree=ModifierCollectionTreeNode(MulConf(ItemConf(f"blocks.{l}.mlp.linear1"),shape=[1,1,model.hidden_size * 4])),
            use_fwd=True,
        ),
    )
    print("RESULT SHAPE", result.shape)
    return result

get_mlp_grads_unjit(model, params, input_ids=dataset_batches[1:2], l=1)


In [ ]:
result['loss'].shape

In [ ]:
result['loss'].shape

In [ ]:
# top_k_doc_idx[new_top_k_idx < batch_size] = new_top_k_idx[new_top_k_idx < batch_size] + i*batch_size
# top_k_doc_idx[new_top_k_idx >= batch_size] = 
k_and_doc_idx, layer_idx, neuron_idx = np.where(new_top_k_idx >= batch_size)
k_and_doc_idx-batch_size

In [ ]:

A[A.argsort(axis=0)[-2:]]

In [ ]:
A[0].shape

In [ ]:
out

In [ ]:
print(log.cache['blocks.1.mlp.gelu'].shape)
print(log.cache['blocks.1.mlp.gelu.identity'].shape)


In [ ]:
keys = set(log.cache.keys())
{k for k in keys if ('mlp' in k) and ('blocks.0' in k) and ('norm' not in k)}

In [ ]:
await ds.show_path_contribution_histogram(model,params,((1,5),),10)